This code goes from Topcoder's winners page, to the problem statement, to the problem detail of each SRM

In [34]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'round_overview'
number = 1
counter = 0
myLinks = []
while(1):
    URL = "http://community.topcoder.com/tc?module=SrmDivisionWins&sc=&sd=&nr=200&sr=" + str(number)
    number += 200
    r = requests .get(URL)
    t = r.text
    
    if ("An error has occurred when attempting to process your request" in t):
        print("There were no results matching your query.")
        print(counter)
        sys.exit()
    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
        
        if("topcoder" in s):
            continue;
        
        # Print the Round overview urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            myLinks.append("http://community.topcoder.com" + s)



There were no results matching your query.
663


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'problem_statement'
number = 1
counter = 0
problemStatements = []
for Link in myLinks:
    URL = Link
    r = requests .get(URL)
    t = r.text

    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
    
        # Print the Problem Statement urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            problemStatements.append("http://community.topcoder.com" + s)


This portion never would finish running on Jupyter, so I ran it on hydra instead. The only change I made was putting the links in a set to strip any duplicates

In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'ProblemDetail'
number = 1
counter = 0
problemDetails = []
for Statement in problemStatements:
    URL = Statement
    r = requests .get(URL)
    t = r.text

    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
    
        # Print the Problem Detail urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            problemDetails.append("http://community.topcoder.com" + s)

This portion has a decent start of stripping the required data from each page. It just makes an array of every word in the text and we can check for words that we are interested in and find the associated values for them.

In [ ]:
#This code doesn't work properly when the problem was used for Div 1 and Div 2
#It needs to store the results from div 1 and then clear the array for Div 2


import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'ProblemDetail'
number = 1
counter = 0
#problemDetails = []

def cleanHtml (html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    return soup .get_text()

f = open('TopcoderDetailsList', 'r')


for line in f:
    URL = line;
    r = requests .get(URL)
    t = r.content
    t = cleanHtml(t).lower()
    #Not sure why, but a few links aren't valid 
    if("an error has occurred when attempting to process your request" in t):
        continue;
    word = ""
    words = []
    #Create an array of the words in the file
    for letter in t:
        if(letter != ' ' and letter != '\n' ):
            word += letter
        else:
            if(word != ""):
                words.append(word)
            word = ""
            
    #All of the respective data will be temporarily stored in arrays
    CategoriesList = [];
    ProblemsSubmitted = []
    ProblemsCorrect = []
    SubmissionAccuracy = []
    ProblemsFailedByChallenge = []
    ProblemsFailedBySystemTest = []
    ChallengeAttemptsMade = []
    ChallengeAccuracy = []
    BestTime = []
    Fastest = []
    AverageCorrectTime = []
    print(URL)
    division = ""
    level = ""
    division2 = ""
    level2 = ""
    
    for i in range(len(words)):
        if(words[i] == "used" and words[i + 1] == "as:"):
            division = words[i + 3]
            level = words[i + 5]
            print("division = " + division)
            print("level = " + level)
            print()
            #Sometimes a single problem will be used in division 1 and 2
            if(words[i + 7] == "division"):
                division2 = words[i + 8]
                level2 = words[i + 10]
                print("division2 = " + division2)
                print("level2 = " + level2)
                print()
        
        if(words[i] == "categories:"):
            j = 1;
            category = ""
            while(words[i + j] != "writer:"):
                if("," in words[i + j]):
                    category += words[i + j]
                    #strip the ','
                    category = category[:-1]
                    CategoriesList.append(category)
                    category = ""
                else:
                    category += words[i + j]
                j += 1;
            CategoriesList.append(category)
            print(CategoriesList)
            
            
        if(words[i] == "problems" and words[i + 1] == "submitted"):
            for j in range(2, 8):
                print(words[i + j])
                ProblemsSubmitted.append(words[i+j])
            print()            
        
        if(words[i] == "problems" and words[i + 1] == "correct"):
            for j in range(2, 8):
                print(words[i + j])
                ProblemsCorrect.append(words[i+j])
            print()
        
        if(words[i] == "submission" and words[i + 1] == "accuracy"):
            for j in range(2, 8):
                print(words[i + j])
                SubmissionAccuracy.append(words[i+j])
            print()
            
        if(words[i] == "problems" and words[i + 1] == "failed" and words[i + 2] == "by" and words[i + 3] == "challenge"):
            i += 2;
            for j in range(2, 8):
                print(words[i + j])
                ProblemsFailedByChallenge.append(words[i+j])
            print()
    
        if(words[i] == "problems" and words[i + 1] == "failed" and words[i + 2] == "by" and words[i + 3] == "system" and words[i + 4] == "test"):
            i += 3;
            for j in range(2, 8):
                print(words[i + j])
                ProblemsFailedBySystemTest.append(words[i+j])
            print()
            
        if(words[i] == "challenge" and words[i + 1] == "attempts" and words[i + 2] == "made"):
            i += 1;
            for j in range(2, 8):
                print(words[i + j])
                ChallengeAttemptsMade.append(words[i+j])
            print()
        
        if(words[i] == "challenge" and words[i + 1] == "accuracy"):
            for j in range(2, 8):
                print(words[i + j])
                ChallengeAccuracy.append(words[i + j])
            print()
    
        #The problem is the best time is blank if no one got it correct.
        if(words[i] == "best" and words[i + 1] == "time"):
            k = 0
            for j in range(2, 8):
                #I check and see if anyone solved the problem
                if(ProblemsCorrect[j - 2] != '0'):
                    print(words[i + j - k])
                    ChallengeAccuracy.append(words[i + j - k])
                else:
                    #If a best time is blank, k helps append the correct word
                    k += 1;
            print()
            
        #These can also have blanks
        
        #-----Important-----
        #At some point it looks like they must've changed the word from "High Scorer" to "Fastest
        if(words[i] == "fastest" or (words[i] == "high" and words[i + 1] == "scorer")):
            if(words[i] == "high"):
                i += 1
            k = 0
            for j in range(1, 7):
                if(ProblemsCorrect[j - 2] != '0'):
                    print(words[i + j - k])
                    Fastest.append(words[i + j - k])
                else:
                    k += 1;
            print()
            
        #These can also have blanks
        if(words[i] == "average" and words[i + 1] == "correct" and words[i + 2] == "time"):
            i += 1;
            k = 0
            for j in range(2, 8):
                if(ProblemsCorrect[j - 2] != '0'):
                    print(words[i + j - k])
                    AverageCorrectTime.append(words[i + j - k])
                else:
                    k += 1;
            print()
       

http://community.topcoder.com/tc?module=ProblemDetail&rd=10804&pm=8415

division = i
level = two

['greedy']
58
286
20
2
0
366

19
87
4
1
0
111

32.76%
30.42%
20.00%
50.00%
0.00
30.33%

13
66
5
0
0
84

26
133
11
1
0
171

33
160
7
2
0
202

39.39%
41.25%
71.43%
0.00%
0.00
41.58%

0:12:43.261
0:09:09.915
0:23:15.874
0:13:20.690
0:09:09.915

andrewzta
kalinov
yarin
jmzero
kalinov

0:31:19.95
0:27:16.788
0:37:23.618
0:13:20.690
0:28:12.600

http://community.topcoder.com/tc?module=ProblemDetail&rd=10804&pm=8414

division = i
level = one

division2 = ii
level2 = two

['simplemath']
56
284
24
2
0
366

45
254
23
2
0
324

80.36%
89.44%
95.83%
100.00%
0.00
88.52%

7
27
1
0
0
35

4
3
0
0
0
7

17
78
11
0
0
106

41.18%
34.62%
9.09%
0.00%
0.00
33.02%

0:02:33.859
0:01:05.0
0:03:34.869
0:11:23.403
0:01:05.0

humblefool
ted
hiltonlange
jmzero
ted

0:20:25.985
0:18:12.955
0:24:35.747
0:23:00.109
0:19:00.377

21
71
7
0
0
99

13
55
3
0
0
71

61.90%
77.46%
42.86%
0.00
0.00
71.72%

8
10
3
0
0
21

0
6
1
0
0
